## Create Tables 

#### Imports 

In [ ]:
from google.cloud.exceptions import NotFound
from google.cloud import bigquery
import datetime
import logging

##### Setup logging 

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

#### Essentials 

In [ ]:
%cd /home/jupyter/bq-sql-agent

In [ ]:
%pwd

In [ ]:
client = bigquery.Client()

#### Create dataset

In [ ]:
dataset_id = f"{client.project}.hotel_reservations"
logger.info(dataset_id)

In [ ]:
try:
    dataset = client.get_dataset(dataset_id)
    logger.info(f"Dataset {dataset_id} already exists!")
except NotFound:
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = "US"
    dataset = client.create_dataset(dataset)
    logger.info(f"Dataset {dataset_id} created.")

#### Create tables 

##### Create `hotels` table

In [ ]:
table_id = f"{dataset_id}.hotels"
logger.info(table_id)

In [ ]:
hotels_schema = [
    bigquery.SchemaField("hotel_id", "INT64", mode="REQUIRED"),
    bigquery.SchemaField("hotel_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("location", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("rating", "FLOAT64", mode="REQUIRED"),
]

In [ ]:
try:
    hotels_table = client.get_table(table_id)
    logger.info(f"Table {table_id} already exists!")
except NotFound:
    hotels_table = bigquery.Table(table_id, schema=hotels_schema)
    hotels_table = client.create_table(hotels_table)  
    logger.info(f"Table {table_id} created.")

In [ ]:
hotels_table.__dict__

##### Create `rooms` table

In [ ]:
table_id = f"{dataset_id}.rooms"
logger.info(table_id)

In [ ]:
rooms_schema = [
    bigquery.SchemaField("room_id", "INT64", mode="REQUIRED"),
    bigquery.SchemaField("hotel_id", "INT64", mode="REQUIRED"),
    bigquery.SchemaField("room_type", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("price_per_night", "FLOAT64", mode="REQUIRED"),
    bigquery.SchemaField("availability", "INT64", mode="REQUIRED"),
]

In [ ]:
try:
    rooms_table = client.get_table(table_id)
    logger.info(f"Table {table_id} already exists!")
except NotFound:
    rooms_table = bigquery.Table(table_id, schema=rooms_schema)
    rooms_table = client.create_table(rooms_table)  
    logger.info(f"Table {table_id} created.")

In [ ]:
rooms_table.__dict__

##### Create `reservations` table

In [ ]:
table_id = f"{dataset_id}.reservations"
logger.info(table_id)

In [ ]:
reservations_schema = [
    bigquery.SchemaField("reservation_id", "INT64", mode="REQUIRED"),
    bigquery.SchemaField("user_id", "INT64", mode="REQUIRED"),
    bigquery.SchemaField("room_id", "INT64", mode="REQUIRED"),
    bigquery.SchemaField("start_date", "DATE", mode="REQUIRED"),
    bigquery.SchemaField("end_date", "DATE", mode="REQUIRED"),
]

In [ ]:
try:
    reservations_table = client.get_table(table_id)
    logger.info(f"Table {table_id} already exists!")
except NotFound:
    reservations_table = bigquery.Table(table_id, schema=reservations_schema)
    reservations_table = client.create_table(reservations_table)  
    logger.info(f"Table {table_id} created.")

In [ ]:
reservations_table.__dict__

#### Populate the tables 

In [ ]:
hotels_data = [
    (1, "Grand Palace", "New York", 4.5),
    (2, "Beachfront Resort", "Miami", 4.2),
    (3, "Mountain View Inn", "Denver", 4.7),
    (4, "City Lights Hotel", "Los Angeles", 4.3),
    (5, "Lakeside Retreat", "Seattle", 4.6),
]

In [ ]:
client.insert_rows(hotels_table, hotels_data)

In [ ]:
rooms_data = [
    # Grand Palace
    (1, 1, "Deluxe", 150.0, 10),
    (2, 1, "Suite", 250.0, 5),
    # Beachfront Resort
    (3, 2, "Deluxe", 180.0, 10),
    (4, 2, "Suite", 280.0, 5),
    # Mountain View Inn
    (5, 3, "Deluxe", 130.0, 10),
    (6, 3, "Suite", 230.0, 5),
    # City Lights Hotel
    (7, 4, "Deluxe", 170.0, 10),
    (8, 4, "Suite", 270.0, 5),
    # Lakeside Retreat
    (9, 5, "Deluxe", 140.0, 10),
    (10, 5, "Suite", 240.0, 5),
]

In [ ]:
client.insert_rows(rooms_table, rooms_data)

In [ ]:
reservations_data = [
    (1, 1, 1, datetime.date(2023, 9, 10), datetime.date(2023, 9, 15)),
    (2, 2, 3, datetime.date(2023, 9, 12), datetime.date(2023, 9, 17)),
]

In [ ]:
client.insert_rows(reservations_table, reservations_data)

Tables take a while to get populated and refreshed in BQ - Run the code below after ~ 2 hours to validate if data is populated

In [ ]:
logger.info(hotels_table.num_rows)

In [ ]:
logger.info(rooms_table.num_rows)

In [ ]:
logger.info(reservations_table.num_rows)